In [0]:
from google.colab import drive
drive.mount('/content/drive')

# Remote SSH

In [0]:
#@title Remote SSH: Copy ngrok authtoken from https://dashboard.ngrok.com/auth { run: "auto" }

import random, string, requests, json, getpass
import time, os

def run():
  password = "123456" #@param {type: "string"}
  authtoken = "2ErRCVGdvrT1MeJSwLc4W_3mgVn8JV9Z3wmTxXuHMYF" #@param {type: "string"}
  sshkey = "ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQC7KbkdOAtC7b3sbTjeP1oNDhp+f2Q3M9Qms8mR6EJi17m+8a1oxvqh3JgZ2vYWo/mV9VJEGb5H86eSS1o6M6dV3+HzCrSbNS83iJJPuhi01oli/AF3gWOZmEzd+KNRxFDy9GnRltxpftgqRLiOhkyKc8mnVxHlvgB4Hihruz0vQfmj/JNLG2xot2EH6V4Ypj5PPFj8kqmM4eJT2MotZz7OU+PTkl6aUI/J/NfjpzsZIm1HlTIaD4xMgypM0Duqu3ft+BcnVtxvez7K4WZJWCVCO0awMTxZp/YnIrIDTn+pQlq6LsZZ/b/MnhBzzRvZryEg1Hrj3pg5TiI0cy4K4tzh rin2401@rinubu" #@param {type: "string"} 

  #Generate root password
  # password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(20))

  print("> Setup sshd")
  ! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null

  print("> Set root password")
  ! echo root:$password | chpasswd
  ! mkdir -p /var/run/sshd
  ! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
  ! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
  ! echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
  ! echo "export LD_LIBRARY_PATH" >> /root/.bashrc
  ! mkdir -p ~/.ssh
  ! echo $sshkey >> ~/.ssh/authorized_keys

  print("> Run sshd")
  get_ipython().system_raw('/usr/sbin/sshd -D &')

  print("> Download ngrok")
  if not os.path.exists('ngrok'):
    !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
    !unzip ngrok-stable-linux-amd64.zip
    !rm ngrok-stable-linux-amd64.zip

  #Ask token
  # print("Copy authtoken from https://dashboard.ngrok.com/auth")
  # authtoken = getpass.getpass()

  print("> Create tunnel")
  get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')
  print("> Wait get address")  
  time.sleep(3)
  host, port = requests.get('http://localhost:4040/api/tunnels').json()['tunnels'][0]['public_url'][6:].split(':')
  print(f'SSH command: ssh -p{port} root@{host}')
  print(f'Root password: {password}')
  print("SSH config:")
  print(f'''Host colab
  HostName {host}
  User root
  Port {port}
  ''')
  
run()

# Jupyter Lab

In [0]:
#@title Jupyter Lab
from IPython.lib import passwd
import os, requests, time

# !apt install tmux
!pip -q install jupyterlab

def lab():
  password = "123456" #@param {type: "string"}
  password = passwd(password)

  print("> Download ngrok")
  if not os.path.exists('ngrok'):
    !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
    !unzip ngrok-stable-linux-amd64.zip
    !rm ngrok-stable-linux-amd64.zip

  print("> Create tunnel")

  get_ipython().system_raw('./ngrok http 8888 &')
  print("> Wait get address")  
  time.sleep(3)
  url = requests.get('http://localhost:4040/api/tunnels').json()['tunnels'][0]['public_url']
  print("Link:", url)
  
  jupyter_running = !jupyter notebook list | grep 8888
  print("> Run jupyter lab")
  if not jupyter_running:
    !mkdir -p /content/.jupyter
    !echo '{ "NotebookApp": { "password": $password } }' > /content/.jupyter/jupyter_notebook_config.json
    get_ipython().system_raw('jupyter lab --ip 0.0.0.0 --no-browser --allow-root &')
  
lab()